# Signal Detection and Processing
- Peak detection, event detection, pattern matching
- Real examples: Heart rate from ECG, Trading signals

In [1]:
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
print('Signal detection module loaded')

Signal detection module loaded


## Peak Detection
**signal.find_peaks()** - Find local maxima
- height: Minimum peak height
- distance: Minimum spacing between peaks
- prominence: How much peak stands out
- width: Peak width requirements

In [2]:
# Signal with multiple peaks
fs = 1000
t = np.linspace(0, 2, 2*fs)
sig = np.sin(2*np.pi*5*t) + 0.5*np.sin(2*np.pi*11*t) + np.random.randn(len(t))*0.1

# Find peaks
peaks, properties = signal.find_peaks(sig, height=0.5, distance=50, prominence=0.3)
print(f'Found {len(peaks)} peaks')
print(f'Peak heights: {properties["peak_heights"][:5]}')

Found 13 peaks
Peak heights: [1.43020741 0.97292306 0.94029236 1.4891195  1.55993096]


## Real Example: Heart Rate from ECG
Detect QRS complexes (heartbeats) in ECG signal
Calculate heart rate and heart rate variability

In [3]:
# Simulate ECG
np.random.seed(42)
fs = 500
duration = 10
t = np.linspace(0, duration, fs*duration)

# Heartbeats at ~70 bpm
heart_rate = 70
beat_interval = 60/heart_rate
beats = np.arange(0, duration, beat_interval)

# Create ECG-like peaks
ecg = np.zeros(len(t))
for beat_time in beats:
    idx = int(beat_time * fs)
    if idx < len(t) - 50:
        ecg[idx:idx+50] += signal.gaussian(50, std=5)

# Add noise
ecg += np.random.randn(len(t)) * 0.1

# Detect peaks
peaks, _ = signal.find_peaks(ecg, height=0.5, distance=fs*0.4)  # Min 0.4s apart

# Calculate heart rate
peak_times = t[peaks]
rr_intervals = np.diff(peak_times)
heart_rates = 60 / rr_intervals
avg_hr = heart_rates.mean()
hrv = heart_rates.std()

print(f'ECG Analysis:')
print(f'  Detected beats: {len(peaks)}')
print(f'  Average heart rate: {avg_hr:.1f} bpm')
print(f'  Heart rate variability: {hrv:.2f} bpm')

AttributeError: module 'scipy.signal' has no attribute 'gaussian'

## Matched Filter Detection
Optimal detector for known signal in noise
Cross-correlate signal with template
Maximum where signal matches

In [ ]:
# Template signal
template = signal.gaussian(50, std=5)

# Noisy signal with embedded template
noise_sig = np.random.randn(1000) * 0.5
noise_sig[300:350] += template  # Embed at position 300
noise_sig[700:750] += template * 0.8  # Another at 700

# Matched filter (cross-correlation)
matched = signal.correlate(noise_sig, template, mode='same')

# Find detections
detections, _ = signal.find_peaks(matched, height=5, distance=100)
print(f'Detected {len(detections)} occurrences')
print(f'Positions: {detections}')

## Real Example: Trading Signal Detection
Detect bullish/bearish patterns in price data
Generate entry/exit signals

In [ ]:
# Stock price simulation
np.random.seed(42)
days = np.arange(252)
price = 100 + 0.1*days + 10*np.sin(2*np.pi*days/50) + np.random.randn(252)*2

# Smooth price
sos = signal.butter(2, 0.1, fs=1, output='sos')
price_smooth = signal.filtfilt(sos[0], sos[1], price)

# Detect local maxima (resistance) and minima (support)
resistance, _ = signal.find_peaks(price_smooth, distance=10, prominence=5)
support, _ = signal.find_peaks(-price_smooth, distance=10, prominence=5)

print(f'Trading levels detected:')
print(f'  Resistance points: {len(resistance)}')
print(f'  Support points: {len(support)}')
print(f'
Buy signals: Price near support')
print(f'Sell signals: Price near resistance')

## Edge Detection
Detect rapid changes (edges) in signals
Useful for: transitions, anomalies, regime changes

In [ ]:
# Signal with step changes
sig_step = np.concatenate([
    np.ones(100) * 1.0,
    np.ones(100) * 2.5,
    np.ones(100) * 1.5,
    np.ones(100) * 3.0
])

# Add noise
sig_step += np.random.randn(400) * 0.1

# Detect edges using derivative
derivative = np.diff(sig_step)
edges = np.where(np.abs(derivative) > 0.5)[0]

print(f'Edge detection:')
print(f'  Detected {len(edges)} edge points')
print(f'  Edge positions: {edges[:10]}')

## Summary

### Key Functions:
```python
# Peak detection
peaks, props = signal.find_peaks(sig, height, distance, prominence, width)

# Matched filter
response = signal.correlate(signal, template, mode='same')

# Edge detection
edges = np.diff(signal)
```

### Applications:
- **Biomedical**: ECG R-peaks, EEG events, breathing detection
- **Trading**: Support/resistance, breakouts, reversals
- **Communications**: Pulse detection, synchronization
- **Quality Control**: Defect detection, anomaly detection

### Best Practices:
✓ **Prefilter**: Remove noise before detection
✓ **Tune parameters**: height, distance, prominence
✓ **Validate**: Check false positives/negatives
✓ **Use prominence**: Better than simple height threshold